# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-05-12 00:04:35] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=35199, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=677960894, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_

[2025-05-12 00:04:42] Attention backend not set. Use fa3 backend by default.
[2025-05-12 00:04:42] Init torch distributed begin.
[2025-05-12 00:04:43] Init torch distributed ends. mem usage=0.00 GB
[2025-05-12 00:04:43] Load weight begin. avail mem=53.74 GB


[2025-05-12 00:04:43] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.57s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.52s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.53s/it]



[2025-05-12 00:04:47] Load weight end. type=Qwen2ForCausalLM, dtype=torch.bfloat16, avail mem=39.30 GB, mem usage=14.44 GB.
[2025-05-12 00:04:47] KV Cache is allocated. #tokens: 20480, K size: 0.55 GB, V size: 0.55 GB
[2025-05-12 00:04:47] Memory pool end. avail mem=37.93 GB


[2025-05-12 00:04:47] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-12 00:04:48] INFO:     Started server process [1508381]
[2025-05-12 00:04:48] INFO:     Waiting for application startup.
[2025-05-12 00:04:48] INFO:     Application startup complete.
[2025-05-12 00:04:48] INFO:     Uvicorn running on http://0.0.0.0:35199 (Press CTRL+C to quit)
[2025-05-12 00:04:48] INFO:     127.0.0.1:52356 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-12 00:04:49] INFO:     127.0.0.1:52364 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-12 00:04:49] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 00:04:51] INFO:     127.0.0.1:52366 - "POST /generate HTTP/1.1" 200 OK
[2025-05-12 00:04:51] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-12 00:04:53] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 00:04:54] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 5.90, #queue-req: 0


[2025-05-12 00:04:54] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, gen throughput (token/s): 95.22, #queue-req: 0


[2025-05-12 00:04:55] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, gen throughput (token/s): 92.00, #queue-req: 0


[2025-05-12 00:04:55] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, gen throughput (token/s): 103.41, #queue-req: 0


...

[2025-05-12 00:04:56] Decode batch. #running-req: 1, #token: 295, token usage: 0.01, gen throughput (token/s): 102.90, #queue-req: 0


[2025-05-12 00:04:57] Decode batch. #running-req: 1, #token: 335, token usage: 0.02, gen throughput (token/s): 95.62, #queue-req: 0


[2025-05-12 00:04:57] Decode batch. #running-req: 1, #token: 375, token usage: 0.02, gen throughput (token/s): 97.66, #queue-req: 0


[2025-05-12 00:04:58] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, gen throughput (token/s): 95.36, #queue-req: 0


[2025-05-12 00:04:58] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, gen throughput (token/s): 96.02, #queue-req: 0
[2025-05-12 00:04:58] INFO:     127.0.0.1:33578 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-12 00:04:58] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 21, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 00:04:59] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 75.16, #queue-req: 0


[2025-05-12 00:04:59] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 92.76, #queue-req: 0


[2025-05-12 00:04:59] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 96.84, #queue-req: 0


[2025-05-12 00:05:00] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 101.77, #queue-req: 0


[2025-05-12 00:05:00] Decode batch. #running-req: 1, #token: 212, token usage: 0.01, gen throughput (token/s): 100.00, #queue-req: 0


[2025-05-12 00:05:01] Decode batch. #running-req: 1, #token: 252, token usage: 0.01, gen throughput (token/s): 101.38, #queue-req: 0


[2025-05-12 00:05:01] Decode batch. #running-req: 1, #token: 292, token usage: 0.01, gen throughput (token/s): 99.83, #queue-req: 0


[2025-05-12 00:05:01] Decode batch. #running-req: 1, #token: 332, token usage: 0.02, gen throughput (token/s): 102.15, #queue-req: 0


[2025-05-12 00:05:02] INFO:     127.0.0.1:33578 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-12 00:05:02] Prefill batch. #new-seq: 1, #new-token: 28, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 00:05:02] Decode batch. #running-req: 1, #token: 42, token usage: 0.00, gen throughput (token/s): 68.67, #queue-req: 0


[2025-05-12 00:05:02] Decode batch. #running-req: 1, #token: 82, token usage: 0.00, gen throughput (token/s): 100.14, #queue-req: 0


[2025-05-12 00:05:03] Decode batch. #running-req: 1, #token: 122, token usage: 0.01, gen throughput (token/s): 99.50, #queue-req: 0


[2025-05-12 00:05:03] Decode batch. #running-req: 1, #token: 162, token usage: 0.01, gen throughput (token/s): 84.27, #queue-req: 0


...

[2025-05-12 00:05:04] Decode batch. #running-req: 1, #token: 242, token usage: 0.01, gen throughput (token/s): 81.38, #queue-req: 0


[2025-05-12 00:05:05] Decode batch. #running-req: 1, #token: 282, token usage: 0.01, gen throughput (token/s): 74.72, #queue-req: 0


[2025-05-12 00:05:05] Decode batch. #running-req: 1, #token: 322, token usage: 0.02, gen throughput (token/s): 72.49, #queue-req: 0


[2025-05-12 00:05:06] Decode batch. #running-req: 1, #token: 362, token usage: 0.02, gen throughput (token/s): 86.05, #queue-req: 0


[2025-05-12 00:05:06] Decode batch. #running-req: 1, #token: 402, token usage: 0.02, gen throughput (token/s): 100.21, #queue-req: 0
[2025-05-12 00:05:06] INFO:     127.0.0.1:33578 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "assistant", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-12 00:05:06] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 00:05:07] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 92.09, #queue-req: 0


[2025-05-12 00:05:07] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 105.07, #queue-req: 0


[2025-05-12 00:05:07] INFO:     127.0.0.1:33578 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "assistant",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-12 00:05:08] Prefill batch. #new-seq: 1, #new-token: 472, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-12 00:05:08] Decode batch. #running-req: 1, #token: 480, token usage: 0.02, gen throughput (token/s): 59.68, #queue-req: 0


[2025-05-12 00:05:08] Decode batch. #running-req: 1, #token: 520, token usage: 0.03, gen throughput (token/s): 101.81, #queue-req: 0


[2025-05-12 00:05:08] Decode batch. #running-req: 1, #token: 560, token usage: 0.03, gen throughput (token/s): 101.11, #queue-req: 0


[2025-05-12 00:05:09] Decode batch. #running-req: 1, #token: 600, token usage: 0.03, gen throughput (token/s): 102.63, #queue-req: 0


[2025-05-12 00:05:09] Decode batch. #running-req: 1, #token: 640, token usage: 0.03, gen throughput (token/s): 100.82, #queue-req: 0


[2025-05-12 00:05:10] Decode batch. #running-req: 1, #token: 680, token usage: 0.03, gen throughput (token/s): 93.59, #queue-req: 0


[2025-05-12 00:05:10] Decode batch. #running-req: 1, #token: 720, token usage: 0.04, gen throughput (token/s): 100.01, #queue-req: 0


[2025-05-12 00:05:10] Decode batch. #running-req: 1, #token: 760, token usage: 0.04, gen throughput (token/s): 101.10, #queue-req: 0


[2025-05-12 00:05:11] INFO:     127.0.0.1:33578 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "assistant",
        "content": "Give me the information and population of the capital of France in the JSON format.",
    },
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-05-12 00:05:11] Prefill batch. #new-seq: 1, #new-token: 22, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-12 00:05:11] Decode batch. #running-req: 1, #token: 36, token usage: 0.00, gen throughput (token/s): 46.00, #queue-req: 0


[2025-05-12 00:05:12] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 92.45, #queue-req: 0


[2025-05-12 00:05:12] Decode batch. #running-req: 1, #token: 116, token usage: 0.01, gen throughput (token/s): 65.97, #queue-req: 0


[2025-05-12 00:05:13] Decode batch. #running-req: 1, #token: 156, token usage: 0.01, gen throughput (token/s): 65.52, #queue-req: 0


[2025-05-12 00:05:14] Decode batch. #running-req: 1, #token: 196, token usage: 0.01, gen throughput (token/s): 77.44, #queue-req: 0


[2025-05-12 00:05:14] Decode batch. #running-req: 1, #token: 236, token usage: 0.01, gen throughput (token/s): 66.00, #queue-req: 0


[2025-05-12 00:05:15] Decode batch. #running-req: 1, #token: 276, token usage: 0.01, gen throughput (token/s): 74.28, #queue-req: 0


[2025-05-12 00:05:15] Decode batch. #running-req: 1, #token: 316, token usage: 0.02, gen throughput (token/s): 63.73, #queue-req: 0


[2025-05-12 00:05:16] Decode batch. #running-req: 1, #token: 356, token usage: 0.02, gen throughput (token/s): 63.61, #queue-req: 0


[2025-05-12 00:05:17] Decode batch. #running-req: 1, #token: 396, token usage: 0.02, gen throughput (token/s): 63.65, #queue-req: 0
[2025-05-12 00:05:17] INFO:     127.0.0.1:55086 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so the user is asking for the information and population of the capital of France in JSON format. Let me break this down.\n\nFirst, I need to identify the capital of France. I know that Paris is the capital, so that\'s straightforward. Now, I should find the most recent population data. I remember that the population of Paris has been growing, but I\'m not sure of the exact number. I think it\'s around 2 million, but I should verify that.\n\nWait, I should check the latest statistics to be accurate. Maybe I can recall that as of 2023, the population was approximately 2,150,000. That seems about right. I should make sure to include this number in the JSON.\n\nNext, I need to structure this information into a JSON format. JSON typically uses key-value pairs, so

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-05-12 00:05:17] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 00:05:17] Decode batch. #running-req: 1, #token: 62, token usage: 0.00, gen throughput (token/s): 63.71, #queue-req: 0


[2025-05-12 00:05:18] Decode batch. #running-req: 1, #token: 102, token usage: 0.00, gen throughput (token/s): 64.70, #queue-req: 0


[2025-05-12 00:05:18] Decode batch. #running-req: 1, #token: 142, token usage: 0.01, gen throughput (token/s): 65.20, #queue-req: 0


[2025-05-12 00:05:19] Decode batch. #running-req: 1, #token: 182, token usage: 0.01, gen throughput (token/s): 69.23, #queue-req: 0


...

[2025-05-12 00:05:21] Decode batch. #running-req: 1, #token: 342, token usage: 0.02, gen throughput (token/s): 103.93, #queue-req: 0


[2025-05-12 00:05:21] Decode batch. #running-req: 1, #token: 382, token usage: 0.02, gen throughput (token/s): 66.20, #queue-req: 0


[2025-05-12 00:05:22] Decode batch. #running-req: 1, #token: 422, token usage: 0.02, gen throughput (token/s): 64.21, #queue-req: 0


[2025-05-12 00:05:22] Decode batch. #running-req: 1, #token: 462, token usage: 0.02, gen throughput (token/s): 64.72, #queue-req: 0


[2025-05-12 00:05:23] Decode batch. #running-req: 1, #token: 502, token usage: 0.02, gen throughput (token/s): 71.47, #queue-req: 0
[2025-05-12 00:05:23] INFO:     127.0.0.1:55088 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-05-12 00:05:23] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 00:05:23] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 00:05:24] Decode batch. #running-req: 3, #token: 74, token usage: 0.00, gen throughput (token/s): 121.35, #queue-req: 0


[2025-05-12 00:05:24] Decode batch. #running-req: 3, #token: 194, token usage: 0.01, gen throughput (token/s): 294.68, #queue-req: 0


[2025-05-12 00:05:24] Decode batch. #running-req: 3, #token: 314, token usage: 0.02, gen throughput (token/s): 295.71, #queue-req: 0


[2025-05-12 00:05:25] Decode batch. #running-req: 3, #token: 434, token usage: 0.02, gen throughput (token/s): 295.26, #queue-req: 0


[2025-05-12 00:05:25] Decode batch. #running-req: 3, #token: 554, token usage: 0.03, gen throughput (token/s): 294.94, #queue-req: 0


[2025-05-12 00:05:25] INFO:     127.0.0.1:40752 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThat's all the information I have.\n\nOkay, so I need to figure out the capital of France. I know that Paris is the capital, but I'm not entirely sure. Let me think about why I think that. I've heard it mentioned a lot, especially in movies and TV shows. People often go there for business or tourism. Also, I remember learning in school that Paris is a major city in France, known for landmarks like the Eiffel Tower and the Louvre Museum. Those places are famous worldwide, which makes me think that Paris is indeed the capital. Maybe I can cross-check this with some other sources or my notes. Wait, I don't have any other information right now, but based on what I know, Paris is the capital of France. I don't recall any other major city in France being referred to as the capital. So, I'm pretty confident that Paris is correct.\n</think>Paris is the capital of F

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-05-12 00:05:26] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 00:05:26] Decode batch. #running-req: 1, #token: 26, token usage: 0.00, gen throughput (token/s): 191.13, #queue-req: 0


[2025-05-12 00:05:26] Decode batch. #running-req: 1, #token: 66, token usage: 0.00, gen throughput (token/s): 104.82, #queue-req: 0


[2025-05-12 00:05:26] Decode batch. #running-req: 1, #token: 106, token usage: 0.01, gen throughput (token/s): 104.26, #queue-req: 0


[2025-05-12 00:05:27] Decode batch. #running-req: 1, #token: 146, token usage: 0.01, gen throughput (token/s): 104.47, #queue-req: 0


...

[2025-05-12 00:05:44] Decode batch. #running-req: 1, #token: 1906, token usage: 0.09, gen throughput (token/s): 103.26, #queue-req: 0


[2025-05-12 00:05:44] Decode batch. #running-req: 1, #token: 1946, token usage: 0.10, gen throughput (token/s): 102.70, #queue-req: 0


[2025-05-12 00:05:45] Decode batch. #running-req: 1, #token: 1986, token usage: 0.10, gen throughput (token/s): 102.48, #queue-req: 0


[2025-05-12 00:05:45] Decode batch. #running-req: 1, #token: 2026, token usage: 0.10, gen throughput (token/s): 105.07, #queue-req: 0


[2025-05-12 00:05:45] INFO:     127.0.0.1:40766 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-05-12 00:05:45] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-12 00:05:45] Decode batch. #running-req: 1, #token: 35, token usage: 0.00, gen throughput (token/s): 98.99, #queue-req: 0


[2025-05-12 00:05:46] Decode batch. #running-req: 1, #token: 75, token usage: 0.00, gen throughput (token/s): 105.58, #queue-req: 0


[2025-05-12 00:05:46] Decode batch. #running-req: 1, #token: 115, token usage: 0.01, gen throughput (token/s): 105.42, #queue-req: 0


[2025-05-12 00:05:47] Decode batch. #running-req: 1, #token: 155, token usage: 0.01, gen throughput (token/s): 104.74, #queue-req: 0


[2025-05-12 00:05:47] Decode batch. #running-req: 1, #token: 195, token usage: 0.01, gen throughput (token/s): 103.07, #queue-req: 0


[2025-05-12 00:05:47] Decode batch. #running-req: 1, #token: 235, token usage: 0.01, gen throughput (token/s): 93.00, #queue-req: 0


[2025-05-12 00:05:48] Decode batch. #running-req: 1, #token: 275, token usage: 0.01, gen throughput (token/s): 100.39, #queue-req: 0
[2025-05-12 00:05:48] INFO:     127.0.0.1:54504 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

[2025-05-12 00:05:48] Child process unexpectedly failed with an exit code 9. pid=1508711
[2025-05-12 00:05:48] Child process unexpectedly failed with an exit code 9. pid=1508645


## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.46s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.38s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.39s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris.

The capital of France is Paris, but I need to know more about it, like its significance as a city, some landmarks, and maybe some history.

Okay, so Paris is the capital of France. It's a very important city in terms of politics, culture, and history. I know it's known for landmarks like the Eiffel Tower, the Louvre Museum, and the Arc de Triomphe. Paris has a rich history going back to ancient times. It was an important city during the Middle Ages and has been a political and cultural center for centuries.

I should probably break down
Prompt: Give me the information of the capital of Germany.
Generated text: 
The capital of Germany is Berlin.

That's all.

How can I improve this?

To make it better, I can add more details, maybe about Berlin's history, culture, or significance in the country.

Alright, let's try to expand on that.

Yes, that would be better. Also, perhaps menti

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its location, economic activities, cultural significance, and artistic contributions.

 This information should be presented in a clear and organized way, perhaps using bullet points or sections.

Additionally, include information about the cultural significance of London's landmarks, such as the London Eye, the Tower of London, and the British Museum. Also, talk about its artistic contributions, like the Tate Modern and the Royal Academy of Arts.

The information should be concise but informative, perhaps around 500 words.

Please ensure that the information is accurate and well-structured.
Alright, so I need to provide information about London as a major global city. The user has specified
Prompt: Please provide information about Paris as a major global city:
Generated text:  its history, culture, economy, and environment.
700-800 words. Please address each of the points in the above question. En

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜Assistant｜>Give me the information and population of the capital of France in the JSON format.<｜end▁of▁sentence｜><｜Assistant｜><think>

Generated text: Okay, so the user is asking for the information and population of the capital of France in JSON format. Hmm, let me break this down. First, I need to figure out what exactly they're looking for. They mentioned both information and population, so I should cover the basic details of the city and its population.

Wait, what's the capital of France? It's Paris, right? I should confirm that to make sure I'm accurate. Yeah, Paris is definitely the capital. Now, they want the population. I remember that population numbers can change over time, so I should check the latest data. I think the population is around 2 million, but I'm not 100% sure. Maybe it's a bit higher now.

Alright, now structuring this in JSON. The user specified the format, so I should present it clearly. I'll need a key for the city and another 

In [19]:
llm.shutdown()